# Live Map Viewer for Person Walkers

This notebook subscribes to MQTT and displays all walking persons on a live map.
Each person appears with their own unique color.

**Usage:**
1. Run all cells to start the map viewer
2. Launch one or more `person_walker.ipynb` notebooks with different names
3. Watch as persons appear and move on the map in real-time!

In [1]:
import importlib
import json
import asyncio
from typing import Any

from IPython.display import display
import simulated_city.maplibre_live as maplibre_live
from simulated_city.config import load_config
from simulated_city.mqtt import MqttConnector

importlib.reload(maplibre_live)
LiveMapLibreMap = maplibre_live.LiveMapLibreMap

In [2]:
# City Hall coordinates (Copenhagen)
CITY_HALL_LNGLAT = (12.5683, 55.6761)

# Create and display the map
m = LiveMapLibreMap(center=CITY_HALL_LNGLAT, zoom=16.5, height="700px")
display(m)
print("✓ Map initialized")

✓ Map initialized


In [3]:
# Connect to MQTT broker
cfg = load_config()
connector = MqttConnector(cfg.mqtt, client_id_suffix="map-viewer")
connector.connect()
if not connector.wait_for_connection(timeout=10.0):
    raise RuntimeError("Failed to connect to MQTT broker")

print("✓ Connected to MQTT broker")

✓ Connected to MQTT broker


In [ ]:
# Track persons we've seen
persons_seen = set()

def on_person_location(client, userdata, message):
    """
    Callback when a person location message arrives.
    
    Expected message format:
    {
        "lng": float,
        "lat": float,
        "color": str,
        "name": str,
        "timestamp": float
    }
    """
    try:
        data = json.loads(message.payload.decode())
        name = data["name"]
        lng = data["lng"]
        lat = data["lat"]
        color = data["color"]
        
        # Create marker if this is the first time we see this person
        if name not in persons_seen:
            persons_seen.add(name)
            print(f"  New person on map: {name} (color: {color})")
        
        # Update marker position with color (move_marker will create if doesn't exist)
        marker_id = f"person-{name}"
        m.move_marker(marker_id, (lng, lat), color=color)
    
    except Exception as e:
        print(f"Error processing message: {e}")

# Subscribe to all person location updates (wildcard +)
connector.client.on_message = on_person_location
connector.client.subscribe("persons/+/location", qos=0)

print("✓ Subscribed to persons/+/location")
print("Waiting for person location updates...\n")

✓ Subscribed to persons/+/location
Waiting for person location updates...



  New person on map: Alice (color: #613ce7)


In [5]:
# Keep the notebook running and display status updates
async def status_updater():
    """Periodically show how many persons are being tracked."""
    while True:
        await asyncio.sleep(10)
        if persons_seen:
            person_list = ", ".join(sorted(persons_seen))
            print(f"  Tracking {len(persons_seen)} person(s): {person_list}")
        else:
            print("  No persons detected yet. Start a person_walker.ipynb!")

status_task = asyncio.create_task(status_updater())
print("✓ Map viewer is running!")
print("Run the next cell to stop.")

✓ Map viewer is running!
Run the next cell to stop.


  Tracking 2 person(s): Alice, Peter
  Tracking 2 person(s): Alice, Peter
  Tracking 2 person(s): Alice, Peter
  Tracking 2 person(s): Alice, Peter
  Tracking 2 person(s): Alice, Peter
  Tracking 2 person(s): Alice, Peter


In [6]:
# Stop the viewer
status_task.cancel()
connector.disconnect()
print("✓ Map viewer stopped.")

Disconnected from MQTT broker (reason=Normal disconnection). Reconnecting...


✓ Map viewer stopped.
